In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.messages import HumanMessage, AIMessage

from langgraph.checkpoint.memory import MemorySaver

from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

from langsmith import utils

import os

In [3]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://eu.api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGCHAIN_PROJECT"]="langchain-agent-test"

os.environ["NVIDIA_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] = "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"

In [4]:
utils.tracing_is_enabled()

True

In [5]:
# using Tavily Search for online search to be used as an agent
search = TavilySearchResults(max_results=5)

search_results = search.invoke("What is the current weather in Barnet, London, United Kingdom")
print(search_results)

tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Barnet', 'region': 'Barnet, Greater London', 'country': 'United Kingdom', 'lat': 51.6445, 'lon': -0.2021, 'tz_id': 'Europe/London', 'localtime_epoch': 1733097356, 'localtime': '2024-12-01 23:55'}, 'current': {'last_updated_epoch': 1733096700, 'last_updated': '2024-12-01 23:45', 'temp_c': 12.3, 'temp_f': 54.1, 'is_day': 0, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/night/296.png', 'code': 1183}, 'wind_mph': 11.4, 'wind_kph': 18.4, 'wind_degree': 221, 'wind_dir': 'SW', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.06, 'precip_in': 0.0, 'humidity': 88, 'cloud': 50, 'feelslike_c': 10.4, 'feelslike_f': 50.8, 'windchill_c': 9.6, 'windchill_f': 49.2, 'heatindex_c': 11.6, 'heatindex_f': 52.9, 'dewpoint_c': 9.5, 'dewpoint_f': 49.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 18.0, 'gust_kph': 28.9}}"}, {'url': 'https://www.timeanddate.com/weather/@3333121', 

In [6]:
# initialise NVIDIA model for chat
model = ChatNVIDIA(model="meta/llama-3.1-405b-instruct")


In [7]:
response = model.invoke([HumanMessage(content = "Hi, my name is Deepak!")])
response.content

'Nice to meet you, Deepak! How are you doing today? Is there something I can help you with or would you like to just chat?'

In [8]:
# explicitly bind the tools to the model
model_with_tools = model.bind_tools(tools)

In [9]:
# tool does not get called with this query
response = model_with_tools.invoke([HumanMessage(content="I live in London, UK")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: London is a fantastic city with a rich history and diverse culture. What can I help you with today?
ToolCalls: []


In [10]:
# tool gets called with this query
response = model_with_tools.invoke([HumanMessage(content = "What is the time in Delhi, India right now?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Delhi India current time'}, 'id': 'chatcmpl-tool-93f9906c357a449c9bd62ea591010507', 'type': 'tool_call'}]


In [11]:
# we are using Langraph to bind the tool with the model
# bind_tools get called internally
agent_executor = create_react_agent(model, tools)

In [12]:
# Search agent does not get invoked
response = agent_executor.invoke({"messages": [HumanMessage(content = "Hi, my name is Deepak!")]})

response["messages"]

[HumanMessage(content='Hi, my name is Deepak!', additional_kwargs={}, response_metadata={}, id='6dd6c960-8aff-4145-be28-505d1122c5f8'),
 AIMessage(content='Nice to meet you, Deepak! Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'Nice to meet you, Deepak! Is there something I can help you with or would you like to chat?', 'token_usage': {'prompt_tokens': 316, 'total_tokens': 339, 'completion_tokens': 23}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-b400723f-c92e-4e53-a41d-b5d4d17dca7b-0', usage_metadata={'input_tokens': 316, 'output_tokens': 23, 'total_tokens': 339}, role='assistant')]

In [13]:
# Search agent gets invoked
response = agent_executor.invoke({"messages":[HumanMessage(content="Which is the highest grossing movie of all time in the world?")]})

response["messages"]

[HumanMessage(content='Which is the highest grossing movie of all time in the world?', additional_kwargs={}, response_metadata={}, id='3464a3a4-6606-446a-b73d-bbc89d8356d6'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-1910e1673bd84ef69ba51cf2945fb7c1', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "highest grossing movie of all time"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'chatcmpl-tool-1910e1673bd84ef69ba51cf2945fb7c1', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "highest grossing movie of all time"}'}}], 'token_usage': {'prompt_tokens': 322, 'total_tokens': 354, 'completion_tokens': 32}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-3b7dc999-260c-4f43-abd5-b59386c314ef-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'highest grossing movie 

In [14]:
# streaming messages
for chunk in agent_executor.stream(
    {"messages":[HumanMessage(content="What is the temperature in Svalbard?")]}
):
    print(chunk)
    print("----")



{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-1d57bca0a8a64d4abda68305fec1aa35', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "Svalbard temperature"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'chatcmpl-tool-1d57bca0a8a64d4abda68305fec1aa35', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "Svalbard temperature"}'}}], 'token_usage': {'prompt_tokens': 317, 'total_tokens': 350, 'completion_tokens': 33}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-f10057b4-388f-45b9-a055-0cffaa68ecdb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Svalbard temperature'}, 'id': 'chatcmpl-tool-1d57bca0a8a64d4abda68305fec1aa35', 'type': 'tool_call'}], usage_metadata={'input_tokens': 317, 'output_tokens': 33, 'total_tokens': 350}, role='assistant')]}}

In [15]:
# adding memory to the agent conversation
memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id":"abc123"}}

for chunk in agent_executor.stream(
    {"messages":[HumanMessage(content="Hi! this is Myra. I live in Edinburgh, Scotland")]}, config
):
    print(chunk)
    print("-----")

{'agent': {'messages': [AIMessage(content="Hello Myra! It's lovely to meet you. Edinburgh is such a beautiful city, steeped in history and culture. What do you like most about living there?", additional_kwargs={}, response_metadata={'role': 'assistant', 'content': "Hello Myra! It's lovely to meet you. Edinburgh is such a beautiful city, steeped in history and culture. What do you like most about living there?", 'token_usage': {'prompt_tokens': 321, 'total_tokens': 355, 'completion_tokens': 34}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-767e3bc7-3413-4def-8505-d4c86c8f2c97-0', usage_metadata={'input_tokens': 321, 'output_tokens': 34, 'total_tokens': 355}, role='assistant')]}}
-----


In [16]:
for chunk in agent_executor.stream(
    {"messages":[HumanMessage(content="What is the current temperature in my city?")]}, config
):
    print(chunk)
    print("-----")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-11fa2de176854760beb246e03c3f05af', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "current temperature in Edinburgh, Scotland"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'chatcmpl-tool-11fa2de176854760beb246e03c3f05af', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "current temperature in Edinburgh, Scotland"}'}}], 'token_usage': {'prompt_tokens': 374, 'total_tokens': 394, 'completion_tokens': 20}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-9d16145a-6311-49df-b9f8-51a3e10586fa-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current temperature in Edinburgh, Scotland'}, 'id': 'chatcmpl-tool-11fa2de176854760beb246e03c3f05af', 'type': 'tool_call'}], usage_metadata={'input_tokens': 37